In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
import time
!pip install torchsummary
from torchsummary import summary

/var/lib/oar/.batch_job_bashrc: line 5: /home/ziwang/.bashrc: No such file or directory


In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [10]:
batch_size = 128
transform = transforms.Compose( # 创建了一个转换组合，将一系列的数据预处理操作组合在一起。这里使用了两个预处理操作
    [transforms.ToTensor(), # 将图像转换为 PyTorch 张量，并将像素值缩放到 [0, 1] 的范围
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # 对图像进行标准化处理，减去均值（0.5）并除以标准差（0.5）
# 创建了 CIFAR-10 数据集的训练集对象。root 参数指定了数据集存储的根目录，train=True 表示加载训练集，
# download=True 表示如果数据集不存在则自动下载，transform=transform 表示应用之前定义的数据预处理操作。
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

Files already downloaded and verified


In [11]:
# 创建了一个训练集数据加载器。trainloader 负责从训练集中加载数据，shuffle=True 表示每个 epoch 都会对数据进行洗牌，
# num_workers=2 表示使用两个子进程来加载数据以加快速度。
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

In [12]:
# 这里分别创建了 CIFAR-10 数据集的测试集对象和测试集数据加载器。
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# 数据类别
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [63]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def count_maxpool_operations(self, input, output, pool_layer, out_channels):
        # input output 的维度均为 N,C,H,W
        kernel_maxpooling = pool_layer.kernel_size
        stride = pool_layer.stride
        padding = pool_layer.padding
        output_height = output.shape[2]
        output_width = output.shape[3]
        out_channels =  output.shape[1]
        # num_max
        # num_max = 0
        print("output_height " + str(output_height))
        print("output_width " + str(output_width))
        print("kernel_maxpooling " + str(kernel_maxpooling**2 -1)) 
        print("out_channels " + str(out_channels))
        num_max = output_height * output_width * (kernel_maxpooling**2 -1) * out_channels
        return num_max

    def count_conv_operations(self, input, output, output_pooled, conv_layer, pool_layer):
        # batch_size = input.size(0)
        out_channels, in_channels = output.size(1), conv_layer.in_channels
        output_height, output_width = output.size(2), output.size(3)
        filter_size = conv_layer.kernel_size[0]
        stride = conv_layer.stride[0]
        padding = conv_layer.padding[0]
        # Compute number of operations for convolution
        print(str(output_height) + "*" +  str(output_width) + "*" + str(in_channels) + "*" + str(filter_size ** 2) + "*" + str(out_channels))
        num_mults = output_height * output_width * in_channels * filter_size ** 2 * out_channels
        num_adds = output_height * output_width * in_channels * filter_size ** 2 * out_channels
        # num_maxs = output_height * output_width * out_channels
        # print("num_mults" + str(num_mults))
        num_maxs = self.count_maxpool_operations(output, output_pooled, pool_layer, out_channels)
        # print("num_maxs" + str(num_maxs))
        total_ops = num_mults + num_adds + num_maxs
        return num_mults, num_adds, num_maxs, total_ops

    def count_operations(self, x):
        conv1_out = self.conv1(x)
        conv1_out_pooled = self.pool(F.relu(conv1_out))  # Apply max pooling after the first convolution
        conv2_out = self.conv2(conv1_out_pooled)
        conv2_out_pooled = self.pool(F.relu(conv2_out))  # Apply max pooling after the first convolution
        # Count operations for convolutional layer 1
        conv1_ops = self.count_conv_operations(x, conv1_out, conv1_out_pooled, self.conv1, self.pool)
        # Count operations for convolutional layer 2
        conv2_ops = self.count_conv_operations(conv1_out_pooled, conv2_out, conv2_out_pooled, self.conv2, self.pool)
        return conv1_ops, conv2_ops        

    def count_fc_operations(self, input, fc_layer):
        # Get the number of input features for the fully connected layer
        in_features = fc_layer.in_features
        # Get the number of output features for the fully connected layer
        out_features = fc_layer.out_features    
        # Compute number of operations for fully connected layer
        # print(str(out_features) + " * " + str(in_features))
        num_mults = out_features * in_features
        num_adds = out_features * in_features
        num_maxs = 0    
        total_ops = num_mults + num_adds
        return num_mults, num_adds, num_maxs, total_ops

    def count_total_operations(self, x):
        conv1_ops, conv2_ops = self.count_operations(x)
        fc1_ops = self.count_fc_operations(x, self.fc1)
        fc2_ops = self.count_fc_operations(x, self.fc2)
        fc3_ops = self.count_fc_operations(x, self.fc3)
        total_ops = sum(op[3] for op in [conv1_ops, conv2_ops, fc1_ops, fc2_ops, fc3_ops])
        return total_ops
# 4-2 实例化网络
net = Net().to(device)

In [64]:
## 5 定义h损失函数
# 这行代码定义了损失函数，即交叉熵损失函数。交叉熵损失函数通常用于多类别分类问题
criterion = nn.CrossEntropyLoss()
# 这行代码定义了优化器，即随机梯度下降（SGD）优化器。SGD是一种常用的优化算法，用于更新神经网络的权重以最小化损失函数。
# lr=0.001指定了学习率，即每次更新时的步长。momentum=0.9是SGD的一个超参数，用于加速SGD在相关方向上前进，并减小波动。
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)    

In [65]:
## 6 定义其他训练所使用的超参数
epoch_nums = 2   #为便于测试可定义为1，实际训练时根据需要改变

In [66]:
## 8 训练模型
# 8-1 记录总时间_起始时间
if torch.cuda.is_available(): torch.cuda.synchronize()
total_time_start = time.time()
# 8-2 记录总操作数、单个epoch操作数、单个batch操作数
total_ops = 0
epoch_train_ops = 0
epoch_evaluate_ops = 0
batch_train_ops = 0
batch_evaluate_ops = 0
# 8-3 遍历epoch执行训练
for epoch in range(epoch_nums):
    print('\nEpoch: %d' % (epoch + 1))
    # 8-3-1 定义统计参数
    net.train()
    sum_loss = 0.0
    correct = 0.0
    total = 0.0
    iteration_num = 0
    # 8-3-2 遍历训练数据集，每次取batch_num个数据
    for i, data in enumerate(trainloader, 0):
        # 8-3-3 准备数据
        length = len(trainloader)
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)        
        optimizer.zero_grad()
        # 8-3-4 统计训练时的操作数，仅需统计一次
        # images.shape[0] 是batch_size,count_total_operations在实际计算时未考虑batch_size
        if(epoch == 0 & i == 0):
            batch_train_ops = net.count_total_operations(inputs)
            batch_train_ops = inputs.shape[0] * batch_train_ops
        # 8-3-5 在数据输入网络前记录时间
        if torch.cuda.is_available(): torch.cuda.synchronize()
        batch_train_start = time.time()
        # 8-3-6 网络计算
        outputs = net(inputs)
        # 8-3-7 在网络处理完成后记录时间
        if torch.cuda.is_available(): torch.cuda.synchronize()
        batch_train_time = time.time() - batch_train_start
        # 8-3-8 计算loss及反向传播
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # 8-3-9 每训练1个batch打印一次loss和准确率等参数
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        # 每秒运算次数
        batch_train_ops_per_second = batch_train_ops / batch_train_time
        print('[Epoch:%d, Iterations:%d] Loss: %.03f | Acc: %.3f%% | Ops: %d| Time: %.6fs | Ops/Sec : %d' % (
            epoch + 1, 
            (i + 1 + epoch * length), 
            sum_loss / (i + 1), 
            100. * correct / total,
            batch_train_ops,
            batch_train_time,
            batch_train_ops_per_second))
        iteration_num += 1
    if(epoch == 0):
        epoch_train_ops = batch_train_ops * iteration_num

    # 8-3-10 每训练完一个epoch测试一下准确率
    # test的时间相对于train较短，因此采用每个epoch统计一次时间
    print("Waiting Test!")
    with torch.no_grad():
        correct = 0
        total = 0
        iteration_num = 0
        # 在数据输入网络前记录时间
        if torch.cuda.is_available(): torch.cuda.synchronize()
        epoch_evaluate_start = time.time()
        for data in testloader:
            net.eval()
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            # 统计训练时的操作数，仅需统计一次
            # images.shape[0] 是batch_size,count_total_operations在实际计算时未考虑batch_size
            if(epoch == 0 & i == 0):
                batch_evaluate_ops = net.count_total_operations(images)
                batch_evaluate_ops = images.shape[0] * batch_evaluate_ops
            outputs = net(images)
            # 取得分最高的那个类 (outputs.data的索引号)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
            iteration_num += 1
        acc = 100. * correct / total
        if(epoch == 0):
            epoch_evaluate_ops = batch_evaluate_ops * iteration_num 
        # 在网络处理完成后记录时间
        if torch.cuda.is_available(): torch.cuda.synchronize()
        epoch_evaluate_time = time.time() - epoch_evaluate_start
        # 每秒运算次数
        epoch_evaluate_ops_per_second = epoch_evaluate_time / epoch_evaluate_time
        print('[Epoch:%d] Validation Acc: %.3f%% | Ops: %d| Time: %.6fs | Ops/Sec : %d' % (
            epoch + 1, 
            acc,
            epoch_evaluate_ops,
            epoch_evaluate_time,
            epoch_evaluate_ops_per_second))


Epoch: 1
28*28*3*25*6
output_height 14
output_width 14
kernel_maxpooling 3
out_channels 6
10*10*6*25*16
output_height 5
output_width 5
kernel_maxpooling 3
out_channels 16
[Epoch:1, Iterations:1] Loss: 2.304 | Acc: 12.500% | Ops: 167445504| Time: 0.000772s | Ops/Sec : 216764613336
28*28*3*25*6
output_height 14
output_width 14
kernel_maxpooling 3
out_channels 6
10*10*6*25*16
output_height 5
output_width 5
kernel_maxpooling 3
out_channels 16
[Epoch:1, Iterations:2] Loss: 2.304 | Acc: 12.891% | Ops: 167445504| Time: 0.000503s | Ops/Sec : 333009647799
28*28*3*25*6
output_height 14
output_width 14
kernel_maxpooling 3
out_channels 6
10*10*6*25*16
output_height 5
output_width 5
kernel_maxpooling 3
out_channels 16
[Epoch:1, Iterations:3] Loss: 2.302 | Acc: 11.979% | Ops: 167445504| Time: 0.000485s | Ops/Sec : 345119089537
28*28*3*25*6
output_height 14
output_width 14
kernel_maxpooling 3
out_channels 6
10*10*6*25*16
output_height 5
output_width 5
kernel_maxpooling 3
out_channels 16
[Epoch:1, It

In [17]:
## 8-4 记录总时间_结束时间
if torch.cuda.is_available(): torch.cuda.synchronize()
total_time_end = time.time() - total_time_start

In [18]:
## 8-5 打印统计结果
total_ops = epoch_nums * (epoch_evaluate_ops + epoch_train_ops)
ops_per_second = total_ops / total_time_end
print('Time elapsed: %.3fs' % total_time_end)
print('total ops: %d' % total_ops)
print('ops/second : %d' % ops_per_second)
print('Finished Training')

Time elapsed: 25.687s
total ops: 84848651712
ops/second : 3303168595
Finished Training


In [31]:
input_size = (3, 32, 32)
net = net.to(device)
summary(net, input_size = input_size)
# Number of Parameters CONVOL1 = out_channels × (in_channels × kernel_size ** 2 + 1) = 6 * (3 * 25 + 1) = 6 * 76 = 456
# Taille de sortie après 1 layer convolution: ((size_input - kernel_size) + 1 // stride) =  (32 - 5) + 1 = 28
# Maxpooling (2 * 2) = Taille de sortie // 2 = 28 // 2 = 14
# MaxPool2d 层不包含可学习的参数 = 0
# Taille de sortie après 2 layer convolution: ((14 - 5) + 1) // 1 = 10
# Number of Parameters CONVOL2 = out_channels × (in_channels × kernel_size ** 2 + 1) = 16 * (6 * 25 + 1) = 2416
# Maxpooling (2 * 2) = Taille de sortie // 2 = 10 // 2 = 5
# Number of Parameters FC1 =(in_channels + 1) × out_channels = ((16 * 5 * 5) + 1) * 120 = 48120
# Number of Parameters FC2 =(in_channels + 1) × out_channels = ((120) + 1) * 84 = 10164
# Number of Parameters FC3 =(in_channels + 1) × out_channels = ((84) + 1) * 10 = 850


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             456
         MaxPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         MaxPool2d-4             [-1, 16, 5, 5]               0
            Linear-5                  [-1, 120]          48,120
            Linear-6                   [-1, 84]          10,164
            Linear-7                   [-1, 10]             850
Total params: 62,006
Trainable params: 62,006
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.31
----------------------------------------------------------------


=================================================================================================

In [32]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [33]:
class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        #self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        #self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        #self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        #self.fc = nn.Linear(512, num_classes)
        #self.fc = nn.Linear(128, num_classes)
        self.fc = nn.Linear(1024, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        #out = self.layer2(out)
        #out = self.layer3(out)
        #out = self.layer4(out)
        #out = F.avg_pool2d(out, 4)
        #out = F.avg_pool2d(out, 16)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


In [34]:
def ResNet18():
    return ResNet(ResidualBlock)

In [35]:
# 创建AlexNet模型 32*32
class My_AlexNet(nn.Module):
    def __init__(self):
        super(My_AlexNet, self).__init__()
        # 特征提取
        self.features = nn.Sequential(
            # 输入通道数为3，因为图片为彩色，三通道
            # 而输出916、卷积核为5*5，步长为1，是由AlexNet模型决定的，后面的都同理
            nn.Conv2d(in_channels=3,out_channels=16,kernel_size=5,stride=1,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2,padding=0),
            nn.Conv2d(in_channels=16,out_channels=48,kernel_size=3,padding=1),
            nn.ReLU(),
            # 添加批处理归一化层,对于batchnorm2d，输入张量的形状应该为（N，C,H，W），
            #  其中num_features参数应该等于C
            nn.BatchNorm2d(48),
            nn.MaxPool2d(kernel_size=3,stride=2,padding=0),
            nn.Conv2d(in_channels=48,out_channels=64,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64,out_channels=48,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2,padding=0)
        )
        # 全连接层
        self.classifier = nn.Sequential(
            # 全连接的第一层，输入肯定是卷积输出的拉平值，即6*6*256
            # 输出是由AlexNet决定的，为4096
            nn.Linear(in_features=3*3*48,out_features=128),
            nn.ReLU(),
            # AlexNet采取了DropOut进行正则，防止过拟合
            nn.Dropout(p=0.5),
            nn.Linear(128,128),
            nn.ReLU(),
            # 最后一层，输出1000个类别，也是我们所说的softmax层
            nn.Linear(128,10)
        )

    # 前向算法
    def forward(self,x):
        x = self.features(x)
        # 不要忘记在卷积--全连接的过程中，需要将数据拉平，之所以从1开始拉平，是因为我们
        # 批量训练，传入的x为[batch（每批的个数）,x(长),x（宽）,x（通道数）]，因此拉平需要从第1（索引，相当于2）开始拉平
        # 变为[batch,x*x*x]
        x = torch.flatten(x, 1)
        result = self.classifier(x)
        return result


In [36]:
# 定义是否使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 参数设置,使得我们能够手动输入命令行参数，就是让风格变得和Linux命令行差不多
# parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
# parser.add_argument('--outf', default='./model/', help='folder to output images and model checkpoints') #输出结果保存路径
# args = parser.parse_args()

# 超参数设置
EPOCH = 80    #遍历数据集次数
# EPOCH = 50
pre_epoch = 0  # 定义已经遍历数据集的次数
BATCH_SIZE = 128      #批处理尺寸(batch_size)
LR = 0.01        #学习率

In [37]:
# 准备数据集并预处理
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  #先四周填充0，在吧图像随机裁剪成32*32
    transforms.RandomHorizontalFlip(),  #图像一半的概率翻转，一半的概率不翻转
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), #R,G,B每层的归一化用到的均值和方差
])

In [38]:
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [39]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train) #训练数据集
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)   #生成一个个batch进行批训练，组成batch的时候顺序打乱取


Files already downloaded and verified


In [40]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)
# Cifar-10的标签
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [41]:
# 模型实例化，在这里更改实际使用到的各种模型
#net = ResNet18().to(device)
net = My_AlexNet().to(device)

# 定义损失函数和优化方式
criterion = nn.CrossEntropyLoss()  #损失函数为交叉熵，多用于多分类问题
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）

# i实例化一个summaryWriter
# writer = SummaryWriter()

In [42]:
if not os.path.exists("./model/"):
    os.makedirs("./model/")
best_acc = 85
print("Start Training Resnet-18!")
with open("acc.txt", "w") as f:
    with open("log.txt", "w")as f2:
        for epoch in range(pre_epoch, EPOCH):
            print('\nEpoch: %d' % (epoch + 1))
            net.train()
            sum_loss = 0.0
            correct = 0.0
            total = 0.0
            for i, data in enumerate(trainloader, 0):
                # 准备数据
                length = len(trainloader)
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                # forward + backward
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                # 每训练1个batch打印一次loss和准确率
                sum_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += predicted.eq(labels.data).cpu().sum()
                print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% '
                      % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))
                f2.write('%03d  %05d |Loss: %.03f | Acc: %.3f%% '
                      % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))
                f2.write('\n')
                f2.flush()
            # writer.add_scalar("Loss/train", sum_loss, epoch)

            # 每训练完一个epoch测试一下准确率
            print("Waiting Test!")
            with torch.no_grad():
                correct = 0
                total = 0
                for data in testloader:
                    net.eval()
                    images, labels = data
                    images, labels = images.to(device), labels.to(device)
                    outputs = net(images)
                    # 取得分最高的那个类 (outputs.data的索引号)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum()

                    # 向tensorboard添加pr曲线
                    # writer.add_pr_curve("pr-curve", labels, predicted)

                print('测试分类准确率为：%.3f%%' % (100 * correct / total))
                acc = 100. * correct / total
                # 将每次测试结果实时写入acc.txt文件中
                print('Saving model......')
                torch.save(net.state_dict(), '%s/net_%03d.pth' % ("./model/", epoch + 1))
                f.write("EPOCH=%03d,Accuracy= %.3f%%" % (epoch + 1, acc))
                f.write('\n')
                f.flush()
                # 记录最佳测试分类准确率并写入best_acc.txt文件中
                if acc > best_acc:
                    f3 = open("best_acc.txt", "w")
                    f3.write("EPOCH=%d,best_acc= %.3f%%" % (epoch + 1, acc))
                    f3.close()
                    best_acc = acc
                # writer.add_scalar("acc/test", acc, epoch)
        print("Training Finished, TotalEPOCH=%d" % EPOCH)

Start Training Resnet-18!

Epoch: 1
[epoch:1, iter:1] Loss: 2.307 | Acc: 9.375% 
[epoch:1, iter:2] Loss: 2.302 | Acc: 11.719% 
[epoch:1, iter:3] Loss: 2.302 | Acc: 11.719% 
[epoch:1, iter:4] Loss: 2.301 | Acc: 11.914% 
[epoch:1, iter:5] Loss: 2.302 | Acc: 11.562% 
[epoch:1, iter:6] Loss: 2.302 | Acc: 11.328% 
[epoch:1, iter:7] Loss: 2.303 | Acc: 10.714% 
[epoch:1, iter:8] Loss: 2.302 | Acc: 11.133% 
[epoch:1, iter:9] Loss: 2.302 | Acc: 11.024% 
[epoch:1, iter:10] Loss: 2.302 | Acc: 10.781% 
[epoch:1, iter:11] Loss: 2.302 | Acc: 11.293% 
[epoch:1, iter:12] Loss: 2.302 | Acc: 11.198% 
[epoch:1, iter:13] Loss: 2.302 | Acc: 11.178% 
[epoch:1, iter:14] Loss: 2.301 | Acc: 11.328% 
[epoch:1, iter:15] Loss: 2.301 | Acc: 11.198% 
[epoch:1, iter:16] Loss: 2.302 | Acc: 11.328% 
[epoch:1, iter:17] Loss: 2.301 | Acc: 11.029% 
[epoch:1, iter:18] Loss: 2.302 | Acc: 10.938% 
[epoch:1, iter:19] Loss: 2.301 | Acc: 10.938% 
[epoch:1, iter:20] Loss: 2.301 | Acc: 10.938% 
[epoch:1, iter:21] Loss: 2.302 | A